In [ ]:
from PIL import Image
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, concatenate
from keras.optimizers import Adam
import os
import matplotlib.pyplot as plt
from keras.losses import binary_crossentropy

import zipfile

data_r = zipfile.ZipFile('train.zip', 'r')
data_r.extractall()

data_r = zipfile.ZipFile('test.zip', 'r')
data_r.extractall()

# Палитра классов для кодирования цветов
palette = {
    0: (60, 16, 152),  # Здание
    1: (132, 41, 246),  # Земля
    2: (110, 193, 228),  # Дорога
    3: (254, 221, 58),  # Растительность
    4: (226, 169, 41),  # Вода
    5: (155, 155, 155)  # Неопределенный
}

invert_palette = {v: k for k, v in palette.items()}
CLASSES = 6

def convert_from_color(arr_3d, palette=invert_palette):
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)
    for i in range(arr_3d.shape[0]):
        for j in range(arr_3d.shape[1]):
            pixel = (arr_3d[i, j, 0], arr_3d[i, j, 1], arr_3d[i, j, 2])
            arr_2d[i, j] = palette.get(pixel, 5)  # По умолчанию 5 для неизвестного
    return arr_2d

def dice_coef(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + 1) / (np.sum(y_true) + np.sum(y_pred) + 1)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def dice_bce_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

def unet_model(image_size, output_classes):
    input_layer = Input(shape=image_size + (3,))
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(input_layer)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Decoder
    up3 = UpSampling2D(size=(2, 2))(conv2)
    up3 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up3)
    merge3 = concatenate([conv1, up3], axis=3)
    conv3 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge3)
    conv3 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    
    # Output Layer
    output_layer = Conv2D(output_classes, 1, activation='softmax')(conv3)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model


def download_data(path):
    data = []
    for path_image in sorted(os.listdir(path=path)):
        image = Image.open(os.path.join(path, path_image))
        data.append(np.array(image))
    return data

X_train = download_data("train/images/")
Y_train = download_data("train/masks/")
X_test = download_data("test/images/")
Y_test = download_data("test/masks/")

X_train_pred = np.array(X_train).reshape([len(X_train)] + list(X_train[0].shape)) / 255
X_test_pred = np.array(X_test).reshape([len(X_test)] + list(X_test[0].shape)) / 255

Y_train_pred = [convert_from_color(Y_train[i][:, :, :3]) for i in range(len(Y_train))]
Y_train_pred = np.array(Y_train_pred)

Y_test_pred = [convert_from_color(Y_test[i][:, :, :3]) for i in range(len(Y_test))]
Y_test_pred = np.array(Y_test_pred)

image_size = X_train[0].shape[:2]
output_classes = CLASSES

model = unet_model(image_size, output_classes)
model.compile(optimizer=Adam(), loss=dice_bce_loss, metrics=[dice_coef])


model.fit(X_train_pred, Y_train_pred, epochs=10, batch_size=16)

# Можно использовать обученную модель для предсказания на тестовых данных
predictions = model.predict(X_test_pred)

# Можно далее обрабатывать предсказания и отображать результаты сегментации

import matplotlib.pyplot as plt

def visualize_segmentation(image, mask, palette):
    mask_rgb = np.zeros_like(image)
    for label, color in palette.items():
        mask_rgb[mask == label] = color
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Оригинальное изображение')
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(mask_rgb)
    plt.title('Маска сегментации')
    plt.axis('off')
    plt.show()

# Пример использования
index = 0  # Индекс изображения для визуализации
sample_image = X_test_pred[index]
sample_mask_pred = np.argmax(predictions[index], axis=-1)
visualize_segmentation(sample_image, sample_mask_pred, palette)